In [13]:
def clean_dataset():
    ## specify sample size "n" of cohort
    n = 1000000
    ## initialize dataframe
    df = pd.DataFrame()
    ## list ID of the "n" subjects
    df['ID'] = list(range(1, n+1))
    ## specify intervention "A" with 0 for 60% of the population and 1 for 40%
    df['time'] = np.random.randint(low=30, high=101, size=df.shape[0])
    df['A'] = 0
    df.loc[df['ID']>df.shape[0]*0.6, 'A'] = 1
    ## specify confounder "C" with 75-25 split with A=0, and 45-55 split with A=1 
    dfA0 = df.loc[df['A']==0, :].reset_index(drop=True)
    dfA0['ID'] = list(range(1, dfA0.shape[0]+1))
    dfA0['C'] = 0
    dfA0.loc[dfA0['ID']>dfA0.shape[0]*0.75, 'C'] = 1
    dfA1 = df.loc[df['A']==1, :].reset_index(drop=True)
    dfA1['ID'] = list(range(1, dfA1.shape[0]+1))
    dfA1['C'] = 0
    dfA1.loc[dfA1['ID']>dfA1.shape[0]*0.45, 'C'] = 1
    df['C'] = pd.concat([dfA0['C'], dfA1['C']], axis=0).reset_index(drop=True)
    ## print(pd.crosstab(df['A'], df['C'], margins=True, normalize=True))
    del dfA0, dfA1
    ## specify logit model for outcome Z with coefficients for A=0.95 and C=0.4
    B0 = -0.55
    B1 = 0.55
    B2 = 0.4
    df['Z'] = B0 + (B1*df['A']) + (B2*df['C']) + np.random.normal(0, 1, df.shape[0])
    df['Y_prob'] = 1 / (1+np.exp(-df['Z']))
    df['Y'] = 0
    df.loc[df['Y_prob']>0.5, 'Y'] = 1
    dfY0 = df.loc[df['Y']==0, :].reset_index(drop=True)
    dfY1 = df.loc[df['Y']==1, :].reset_index(drop=True)
    dfY1 = dfY1.sample(n=int(dfY1.shape[0]*0.1), random_state=1005, replace=False).reset_index(drop=True)
    df = pd.concat([dfY0, dfY1], axis=0).reset_index(drop=True)
    
    ## print crosstabs from dataset df
    print('The crosstab of intervention A and outcome Y :')
    print(pd.crosstab(df['A'], df['Y'], margins=True, normalize=True))
    print('\n')
    print('The crosstab of confounder C and outcome Y :')
    print(pd.crosstab(df['C'], df['Y'], margins=True, normalize=True))
    print('\n')
    print('The counts of outcome Y :')
    print(df['Y'].value_counts(normalize=True))
    print('\n')    
    return(df)


class CC_dataset:
    
    def __init__(self):
        self.df = None
        self.df_CaseControl_IDS = None
        self.df_CaseControl_RSS = None
        self.df_CaseControl_CDS = None
        self.df_CaseBase = None
        self.df_CaseCohort = None

    def recover_CC_datasets(self, df):
        self.df = df
        self.df_CaseControl_IDS = CC_dataset.create_CC_dataset(self, samp='Incidence Density Sampling')
        self.df_CaseControl_RSS = CC_dataset.create_CC_dataset(self, samp='Risk Set Sampling')
        self.df_CaseControl_CDS = CC_dataset.create_CC_dataset(self, samp='Cumulative Density Sampling')
        self.df_CaseBase = CC_dataset.create_CC_dataset(self, samp='Case-Base Sampling')
        self.df_CaseCohort = CC_dataset.create_CC_dataset(self, samp='Case Cohort Sampling')


    def create_CC_dataset(self, samp='Incidence Density Sampling'):
        df = self.df
        
        if(samp=='Incidence Density Sampling'):
            df_C0 = df.loc[df['C']==0, :].reset_index(drop=True)
            df_C1 = df.loc[df['C']==1, :].reset_index(drop=True)
            df_C0['weight'] = df_C0['time'] / df_C0['time'].sum()
            df_C1['weight'] = df_C1['time'] / df_C1['time'].sum()
            df_cases_C0 = df_C0.loc[df_C0['Y']==1, :].reset_index(drop=True)
            df_cases_C1 = df_C1.loc[df_C1['Y']==1, :].reset_index(drop=True)
            df_cases_C0['cases'] = 1
            df_cases_C1['cases'] = 1
            df_controls_C0 = df_C0.sample(n=df_cases_C0.shape[0]*3, random_state=1567, replace=True, weights=df_C0['weight']).reset_index(drop=True)
            df_controls_C1 = df_C1.sample(n=df_cases_C1.shape[0]*3, random_state=1567, replace=True, weights=df_C1['weight']).reset_index(drop=True)
            df_controls_C0['cases'] = 0
            df_controls_C1['cases'] = 0
            df_CC = pd.concat([df_cases_C0, df_cases_C1, df_controls_C0, df_controls_C1], axis=0).reset_index(drop=True)
   
        if(samp=='Risk Set Sampling'):
            df_C0 = df.loc[df['C']==0, :].reset_index(drop=True)
            df_C1 = df.loc[df['C']==1, :].reset_index(drop=True)
            df_cases_C0 = df_C0.loc[df_C0['Y']==1, :].reset_index(drop=True)
            df_cases_C1 = df_C1.loc[df_C1['Y']==1, :].reset_index(drop=True)
            df_cases_C0['cases'] = 1
            df_cases_C1['cases'] = 1
            time_vec_C0 = list(set(df_cases_C0['time']))
            time_vec_C1 = list(set(df_cases_C1['time']))
        
            for i in range(0, len(time_vec_C0)):
                time = time_vec_C0[i]
                df_C0_sub = df_C0.loc[df_C0['time']>=time, :].reset_index(drop=True)
                df_controls_C0_sub = df_C0_sub.sample(n=(5*df_cases_C0.loc[df_cases_C0['time']==time, :].reset_index(drop=True).shape[0]), random_state=1567, replace=True).reset_index(drop=True)
                if(i==0):
                    df_controls_C0 = df_controls_C0_sub
                else:
                    df_controls_C0 = pd.concat([df_controls_C0, df_controls_C0_sub], axis=0).reset_index(drop=True)
                del time, df_C0_sub, df_controls_C0_sub
        
            for i in range(0, len(time_vec_C1)):
                time = time_vec_C1[i]
                df_C1_sub = df_C1.loc[df_C1['time']>=time, :].reset_index(drop=True)
                df_controls_C1_sub = df_C1_sub.sample(n=(5*df_cases_C1.loc[df_cases_C1['time']==time, :].reset_index(drop=True).shape[0]), random_state=1567, replace=True).reset_index(drop=True)
                if(i==0):
                    df_controls_C1 = df_controls_C1_sub
                else:
                    df_controls_C1 = pd.concat([df_controls_C1, df_controls_C1_sub], axis=0).reset_index(drop=True)
                del time, df_C1_sub, df_controls_C1_sub
        
            df_controls_C0['cases'] = 0
            df_controls_C1['cases'] = 0
            df_CC = pd.concat([df_cases_C0, df_cases_C1, df_controls_C0, df_controls_C1], axis=0).reset_index(drop=True)

        if(samp=='Cumulative Density Sampling'):
            df_C0 = df.loc[df['C']==0, :].reset_index(drop=True)
            df_C1 = df.loc[df['C']==1, :].reset_index(drop=True)
            df_cases_C0 = df_C0.loc[df_C0['Y']==1, :].reset_index(drop=True)
            df_cases_C1 = df_C1.loc[df_C1['Y']==1, :].reset_index(drop=True)
            df_cases_C0['cases'] = 1
            df_cases_C1['cases'] = 1
            df_controls_C0 = df_C0.loc[df_C0['Y']==0, :].reset_index(drop=True)
            df_controls_C1 = df_C1.loc[df_C1['Y']==0, :].reset_index(drop=True)
            df_controls_C0 = df_controls_C0.sample(n=df_cases_C0.shape[0]*3, random_state=1567, replace=True).reset_index(drop=True)
            df_controls_C1 = df_controls_C1.sample(n=df_cases_C1.shape[0]*3, random_state=1567, replace=True).reset_index(drop=True)
            df_controls_C0['cases'] = 0
            df_controls_C1['cases'] = 0
            df_CC = pd.concat([df_cases_C0, df_cases_C1, df_controls_C0, df_controls_C1], axis=0).reset_index(drop=True)
        
        if(samp=='Case-Base Sampling'):
            df_C0 = df.loc[df['C']==0, :].reset_index(drop=True)
            df_C1 = df.loc[df['C']==1, :].reset_index(drop=True)
            df_cases_C0 = df_C0.loc[df_C0['Y']==1, :].reset_index(drop=True)
            df_cases_C1 = df_C1.loc[df_C1['Y']==1, :].reset_index(drop=True)
            df_cases_C0['cases'] = 1
            df_cases_C1['cases'] = 1
            df_controls_C0 = df_C0.sample(n=df_cases_C0.shape[0]*3, random_state=1567, replace=True).reset_index(drop=True)
            df_controls_C1 = df_C1.sample(n=df_cases_C1.shape[0]*3, random_state=1567, replace=True).reset_index(drop=True)
            df_controls_C0['cases'] = 0
            df_controls_C1['cases'] = 0
            df_CC = pd.concat([df_cases_C0, df_cases_C1, df_controls_C0, df_controls_C1], axis=0).reset_index(drop=True)
    
        if(samp=='Case Cohort Sampling'):
            df_C0 = df.loc[df['C']==0, :].reset_index(drop=True)
            df_C1 = df.loc[df['C']==1, :].reset_index(drop=True)
            df_cases_C0 = df_C0.loc[df_C0['Y']==1, :].reset_index(drop=True)
            df_cases_C1 = df_C1.loc[df_C1['Y']==1, :].reset_index(drop=True)
            df_sub_cohort = df.sample(n=3*(df_cases_C0.shape[0] + df_cases_C1.shape[0]), random_state=1005, replace=True).reset_index(drop=True)
        
            df_cases_C0['weight'] = None
            df_cases_C1['weight'] = None
                
            df_cases_C0['cases'] = 1
            df_cases_C1['cases'] = 1
            time_vec_C0 = list(set(df_cases_C0['time']))
            time_vec_C1 = list(set(df_cases_C1['time']))
    
            for i in range(0, len(time_vec_C0)):
                time = time_vec_C0[i]
                df_controls_C0_sub = df_sub_cohort.loc[(df_sub_cohort['time']>=time) & (df_sub_cohort['C']==0), :].reset_index(drop=True)
                df_controls_C0_sub['weight'] = df_cases_C0.loc[df_cases_C0['time']==time, :].reset_index(drop=True).shape[0] / df_controls_C0_sub.shape[0]
                df_cases_C0.loc[df_cases_C0['time']==time, 'weight'] = df_cases_C0.loc[df_cases_C0['time']==time, :].reset_index(drop=True).shape[0] / df_controls_C0_sub.shape[0]     
                if(i==0):
                    df_controls_C0 = df_controls_C0_sub
                else:
                    df_controls_C0 = pd.concat([df_controls_C0, df_controls_C0_sub], axis=0).reset_index(drop=True)
                del time, df_controls_C0_sub
        
            for i in range(0, len(time_vec_C1)):
                time = time_vec_C1[i]
                df_controls_C1_sub = df_sub_cohort.loc[(df_sub_cohort['time']>=time) & (df_sub_cohort['C']==1), :].reset_index(drop=True)
                df_controls_C1_sub['weight'] = df_cases_C1.loc[df_cases_C1['time']==time, :].reset_index(drop=True).shape[0] / df_controls_C1_sub.shape[0]
                df_cases_C1.loc[df_cases_C1['time']==time, 'weight'] = df_cases_C1.loc[df_cases_C1['time']==time, :].reset_index(drop=True).shape[0] / df_controls_C1_sub.shape[0]
                if(i==0):
                    df_controls_C1 = df_controls_C1_sub
                else:
                    df_controls_C1 = pd.concat([df_controls_C1, df_controls_C1_sub], axis=0).reset_index(drop=True)
                del time, df_controls_C1_sub
        
            df_controls_C0['cases'] = 0
            df_controls_C1['cases'] = 0
               
            df_CC_C0 = pd.concat([df_cases_C0, df_controls_C0], axis=0).reset_index(drop=True)
            df_CC_C1 = pd.concat([df_cases_C1, df_controls_C1], axis=0).reset_index(drop=True)
            df_CC_C0['weight'] = df_CC_C0['weight'] / df_CC_C0['weight'].sum()
            df_CC_C1['weight'] = df_CC_C1['weight'] / df_CC_C1['weight'].sum()
            df_CC = pd.concat([df_CC_C0, df_CC_C1], axis=0).reset_index(drop=True)

        return(df_CC)

In [14]:
####################
## Load Libraries ##
####################
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
np.random.seed(10815657)

In [15]:
## get cleaned dataset
df = clean_dataset()
## specify df_CC class object
df_CC = CC_dataset()
## recover all Case-Control datasets
df_CC.recover_CC_datasets(df)

The crosstab of intervention A and outcome Y :
Y           0         1       All
A                                
0    0.658697  0.032405  0.691103
1    0.270851  0.038046  0.308897
All  0.929548  0.070452  1.000000


The crosstab of confounder C and outcome Y :
Y           0         1       All
C                                
0    0.668402  0.035886  0.704288
1    0.261146  0.034566  0.295712
All  0.929548  0.070452  1.000000


The counts of outcome Y :
0    0.929548
1    0.070452
Name: Y, dtype: float64




In [16]:
#####################################################
## Case Control study - Incidence Density Sampling ##
#####################################################
## print out univariate and multivariate poisson models with time-lag
PRtl_multi = smf.glm("Y ~ A + C", data=df_CC.df, family=sm.families.Poisson(), offset=(np.log(df['time']))).fit()
print(PRtl_multi.summary())
print('\n\n')
## Case Control study, Incidence Density Sampling
LR_CC_IDS = smf.glm("cases ~ A + C", data=df_CC.df_CaseControl_IDS, family=sm.families.Binomial()).fit()
print(LR_CC_IDS.summary())
print('\n\n')
print("The Cohort Analysis Incidence Rate Ratio (IRR) is " + str(np.exp(PRtl_multi.params[1])))
print("The Case-Control with Incidence Density Sampling Odds Ratio (OR) is " + str(np.exp(LR_CC_IDS.params[1])))

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      Y   No. Observations:               611967
Model:                            GLM   Df Residuals:                   611964
Model Family:                 Poisson   Df Model:                            2
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.5306e+05
Date:                Fri, 13 Nov 2020   Deviance:                   2.1989e+05
Time:                        02:33:33   Pearson chi2:                 6.41e+05
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -7.3974      0.008   -941.971      0.0

In [17]:
############################################
## Case Control study - Risk Set Sampling ##
############################################
## print out univariate and multivariate poisson models with time-lag
PRtl_multi = smf.glm("Y ~ A + C + time", data=df_CC.df, family=sm.families.Poisson(), offset=(np.log(df['time']))).fit()
print(PRtl_multi.summary())
print('\n\n')
## Case Control study
LR_CC_RSS = smf.glm("cases ~ A + C + time", data=df_CC.df_CaseControl_RSS, family=sm.families.Binomial()).fit()
print(LR_CC_RSS.summary())
print('\n\n')
print("The Cohort Analysis Incidence Rate Ratio (IRR) is " + str(np.exp(PRtl_multi.params[1])))
print("The Case-Control with Risk-Set Sampling Odds Ratio (OR) is " + str(np.exp(LR_CC_RSS.params[1])))

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      Y   No. Observations:               611967
Model:                            GLM   Df Residuals:                   611963
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.5085e+05
Date:                Fri, 13 Nov 2020   Deviance:                   2.1548e+05
Time:                        02:33:46   Pearson chi2:                 5.70e+05
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.3004      0.018   -359.503      0.0

In [18]:
######################################################
## Case Control study - Cumulative Density Sampling ##
######################################################
## print out univariate and multivariate logistic models
LR_multi = smf.glm("Y ~ A + C", data=df_CC.df, family=sm.families.Binomial()).fit()
print(LR_multi.summary())
print('\n\n')
## Case Control study
LR_CC_CDS = smf.glm("cases ~ A + C", data=df_CC.df_CaseControl_CDS, family=sm.families.Binomial()).fit()
print(LR_CC_CDS.summary())
print('\n\n')
print("The Cohort Analysis Odds Ratio (OR) is " + str(np.exp(LR_multi.params[1])))
print("The Case-Control with Cumulative Density Sampling Odds Ratio (OR) is " + str(np.exp(LR_CC_CDS.params[1])))

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      Y   No. Observations:               611967
Model:                            GLM   Df Residuals:                   611964
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.4864e+05
Date:                Fri, 13 Nov 2020   Deviance:                   2.9728e+05
Time:                        02:34:11   Pearson chi2:                 6.12e+05
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.1913      0.008   -394.997      0.0

In [19]:
#####################
## Case-Base study ##
#####################
## print out univariate and multivariate logistic models
PR_multi = smf.glm("Y ~ A + C", data=df_CC.df, family=sm.families.Poisson()).fit()
print(PR_multi.summary())
print('\n\n')
## Case-Base study
LR_CB = smf.glm("cases ~ A + C", data=df_CC.df_CaseBase, family=sm.families.Binomial()).fit()
print(LR_CB.summary())
print('\n\n')
print("The Cohort Analysis Risk Ratio (RR) is " + str(np.exp(PR_multi.params[1])))
print("The Case-Base Analysis Odds Ratio (OR) is " + str(np.exp(LR_CB.params[1])))

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      Y   No. Observations:               611967
Model:                            GLM   Df Residuals:                   611964
Model Family:                 Poisson   Df Model:                            2
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.5079e+05
Date:                Fri, 13 Nov 2020   Deviance:                   2.1535e+05
Time:                        02:34:25   Pearson chi2:                 5.68e+05
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.2234      0.008   -410.439      0.0

In [22]:
######################
## Case Chort Study ##
######################
## print out univariate and multivariate poisson models with time-lag
PRtl_multi = smf.glm("Y ~ A + C + time", data=df_CC.df, family=sm.families.Poisson(), offset=(np.log(df['time']))).fit()
print(PRtl_multi.summary())
print('\n\n')
## Case Cohort study
clf = LogisticRegression(penalty='none', random_state=0).fit(df_CC.df_CaseCohort[['A', 'C', 'time']].copy(), df_CC.df_CaseCohort['Y'].copy(), sample_weight=df_CC.df_CaseCohort['weight'])
print(clf.coef_)
print('\n\n')
print("The Cohort Analysis Incidence Rate Ratio (IRR) is " + str(np.exp(PR_multi.params[1])))
print("The Case-Cohort Analysis Odds Ratio (OR) is " + str(np.exp(clf.coef_[0][0])))

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      Y   No. Observations:               611967
Model:                            GLM   Df Residuals:                   611963
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.5085e+05
Date:                Fri, 13 Nov 2020   Deviance:                   2.1548e+05
Time:                        02:37:57   Pearson chi2:                 5.70e+05
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.3004      0.018   -359.503      0.0